In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte

In [ ]:
BLOCK_SIZE = 32

In [ ]:
def compute_flow_magnitude(flow_block):
    dx = flow_block[..., 0]
    dy = flow_block[..., 1]
    mag = np.sqrt(dx**2 + dy**2)
    return np.mean(mag)

def extract_of_features(ref_frame, tar_frame, block_size=32):
    h, w = ref_frame.shape
    features = []

    flow = cv2.calcOpticalFlowFarneback(
        prev=ref_frame, next=tar_frame,
        flow=None, pyr_scale=0.5, levels=3, winsize=15,
        iterations=3, poly_n=5, poly_sigma=1.2, flags=0
    )

    if block_size == 0:
        avg_mag = compute_flow_magnitude(flow)
        features.append({'block_y': 0, 'block_x': 0, 'of': avg_mag})
        return np.array([[avg_mag]])

    for y in range(0, h - block_size + 1, block_size):
        for x in range(0, w - block_size + 1, block_size):
            flow_block = flow[y:y+block_size, x:x+block_size, :]
            mag = compute_flow_magnitude(flow_block)
            print(f"OF magnitude at block (y={y}, x={x}) = {mag:.4f}")
            features.append(mag)

    return np.array(features).reshape(h // block_size, w // block_size)

In [ ]:
frame_dir = Path("dataset/DAVIS/curling")
frame_paths = sorted(frame_dir.glob("*.jpg"))

gray_frames = []
for path in frame_paths:
    image = imread(path)
    gray = rgb2gray(image) if image.ndim == 3 else image
    gray = img_as_ubyte(gray)
    gray_frames.append(gray)

ref_frame, tar_frame = gray_frames[0], gray_frames[5]
of_map = extract_of_features(ref_frame, tar_frame, block_size=BLOCK_SIZE)

plt.figure(figsize=(6, 5))
plt.imshow(of_map, cmap='hot', interpolation='nearest')
plt.title(f"OF Magnitude Heatmap (block_size={BLOCK_SIZE})")
plt.colorbar()
plt.axis("off")
plt.tight_layout()
plt.show()